In [ ]:
! curl -fsSL https://ollama.com/install.sh | sed 's#https://ollama.com/download#https://github.com/jmorganca/ollama/releases/download/v0.1.27#' | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
! ollama serve

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model from a Modelfile
  show        Show information for a model
  run         Run a model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.


In [ ]:
!pip install ngrok
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.1 MB/s eta 0:00:00


In [ ]:
import threading
import time
import os
import asyncio
from pyngrok import ngrok
import threading
import queue
import time
from threading import Thread

# Get your ngrok token from your ngrok account:
# https://dashboard.ngrok.com/get-started/your-authtoken
token="auth token here "
ngrok.set_auth_token(token)

# set up a stoppable thread (not mandatory, but cleaner if you want to stop this later
class StoppableThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

def start_ngrok(q, stop_event):
    try:
        # Start an HTTP tunnel on the specified port
        public_url = ngrok.connect(11434)
        # Put the public URL in the queue
        q.put(public_url)
        # Keep the thread alive until stop event is set
        while not stop_event.is_set():
            time.sleep(1)  # Adjust sleep time as needed
    except Exception as e:
        print(f"Error in start_ngrok: {e}")

In [ ]:
# Create a queue to share data between threads
url_queue = queue.Queue()

# Start ngrok in a separate thread
ngrok_thread = StoppableThread(target=start_ngrok, args=(url_queue, StoppableThread.is_stopped))
ngrok_thread.start()

In [ ]:
# Wait for the ngrok tunnel to be established
while True:
    try:
        public_url = url_queue.get()
        if public_url:
            break
        print("Waiting for ngrok URL...")
        time.sleep(1)
    except Exception as e:
        print(f"Error in retrieving ngrok URL: {e}")

print("Ngrok tunnel established at:", public_url)

Ngrok tunnel established at: NgrokTunnel: "https://9e2a-34-67-48-221.ngrok-free.app" -> "http://localhost:11434"


In [ ]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [ ]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve
Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIE2SYfCf1q/NU4zR+TTI+ftFXDPQzIqPw3s9uFSij8hh

time=2024-04-05T10:13:19.916Z level=INFO source=images.go:710 msg="total blobs: 0"
time=2024-04-05T10:13:19.916Z level=INFO source=images.go:717 msg="total unused blobs removed: 0"
time=2024-04-05T10:13:19.917Z level=INFO source=routes.go:1019 msg="Listening on 127.0.0.1:11434 (version 0.1.27)"
time=2024-04-05T10:13:19.917Z level=INFO source=payload_common.go:107 msg="Extracting dynamic libraries..."


In [ ]:

! export OLLAMA_HOST=https://9e2a-34-67-48-221.ngrok-free.app


In [ ]:
!ollama pull llama2

[GIN] 2024/04/05 - 10:17:51 | 200 |      26.472µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ time=2024-04-05T10:17:52.783Z level=INFO source=download.go:136 msg="downloading 8934d96d3f08 in 39 100 MB part(s)"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏ 1.1 MB/3.8 GB                  pullin

In [ ]:
! curl -X POST https://9e2a-34-67-48-221.ngrok-free.app/api/generate -d '{ "model": "gemma", "prompt":"hello buddy respond with hi" , "stream": false }'

[GIN] 2024/04/05 - 11:12:25 | 200 |  498.431684ms |    34.67.48.221 | POST     "/api/generate"
{"model":"gemma","created_at":"2024-04-05T11:12:25.225372763Z","response":"Hello, hello, and greetings from the","done":true,"context":[106,1645,108,17534,35847,8702,675,5827,107,108,106,2516,108,4521,235269,25612,235269,578,78107,774,573,107,108],"total_duration":497891556,"load_duration":464504,"prompt_eval_count":10,"prompt_eval_duration":261914000,"eval_count":9,"eval_duration":234921000}

new book

In [ ]:
! curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama pull mistral

[GIN] 2024/04/05 - 11:16:16 | 200 |      33.993µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ time=2024-04-05T11:16:18.019Z level=INFO source=download.go:136 msg="downloading e8a35b5937a5 in 42 100 MB part(s)"
pulling manifest ⠙ pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pullin

In [ ]:
!ollama serve

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAICwa0+yfYHfEg8sBFHsI+aJlWpyGi2xPW20NqmLVfBcy

time=2024-04-05T10:07:27.795Z level=INFO source=images.go:804 msg="total blobs: 0"
time=2024-04-05T10:07:27.795Z level=INFO source=images.go:811 msg="total unused blobs removed: 0"
time=2024-04-05T10:07:27.795Z level=INFO source=routes.go:1118 msg="Listening on 127.0.0.1:11434 (version 0.1.30)"
time=2024-04-05T10:07:27.796Z level=INFO source=payload_common.go:113 msg="Extracting dynamic libraries to /tmp/ollama272116667/runners ..."
time=2024-04-05T10:07:35.822Z level=INFO source=payload_common.go:140 msg="Dynamic LLM libraries [cpu_avx cuda_v11 cpu_avx2 rocm_v60000 cpu]"
time=2024-04-05T10:07:35.822Z level=INFO source=gpu.go:115 msg="Detecting GPU type"
time=2024-04-05T10:07:35.822Z level=INFO source=gpu.go:265 msg="Searching for GPU management library libcudart.so*"
time=2024-04-05T10:07:35.899Z level=INFO so

In [ ]:
! curl -X POST https://9e2a-34-67-48-221.ngrok-free.app/api/generate -d '{ "model": "mistral", "prompt":"hello buddy respond with hi" , "stream": false }'

[GIN] 2024/04/05 - 11:18:39 | 200 |  778.799139ms |    34.67.48.221 | POST     "/api/generate"
{"model":"mistral","created_at":"2024-04-05T11:18:39.687927997Z","response":"\n Question: What is the third smallest value in -3, 0.2, 0.1?\n Answer: 0.2","done":true,"context":[733,16289,28793,28705,6312,28709,26123,9421,395,12014,733,28748,16289,28793,13,22478,28747,1824,349,272,4008,22341,1192,297,387,28770,28725,28705,28734,28723,28750,28725,28705,28734,28723,28740,28804,13,26307,28747,28705,28734,28723,28750],"total_duration":778307946,"load_duration":510206,"prompt_eval_duration":40001000,"eval_count":31,"eval_duration":737001000}